In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



**Creating a unique mapping from characters to indices:**

In [ ]:
vocab = sorted(set(text))

In [ ]:
char2idx = {u:i for i , u in enumerate(vocab)}

idx2char = np.array(vocab)

In [ ]:
def text_to_int(text):
   return np.array([char2idx[c] for c in text])
   

In [ ]:
text_as_int = text_to_int(text)

In [ ]:
#Checking the results:
print("Text:", text[:20])
print("Encoded:", text_as_int[:20])


Text: First Citizen:
Befor
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()

  except:
    pass

  return ''.join(idx2char[ints])


In [ ]:
print(int_to_text(text_as_int[:20]))

First Citizen:
Befor


**Training Examples:**

In [ ]:
seq_length = 100 #length of sequence for a training example

examples_epoch = len(text)//(seq_length+1)



In [ ]:
#creating training examples / target

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
#We use the batch method to turn this stream of characters into batchs of the desire length
sequences = char_dataset.batch(seq_length+1 , drop_remainder=True)

In [ ]:
#we split the sequences into input and output 
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]

  return input_text , target_text
  

In [ ]:
dataset = sequences.map(split_input_target)


In [ ]:
for x,y in dataset.take(2):
  print('\n\n Example \n\n')
  print("INPUT")
  print(int_to_text(x))
  print("\n OUTPUT \n")
  print(int_to_text(y))
  




 Example 


INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

 OUTPUT 

irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


 Example 


INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

 OUTPUT 

re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)


In [ ]:
def build_model(vocab_size,dim,rnn_units,batch_size):
  model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,batch_input_shape=[batch_size , None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences =True, stateful=True, recurrent_initializer ='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
                               ])
  return model 

In [ ]:
model = build_model(VOCAB_SIZE,DIM,RNN_UNITS,BATCH_SIZE)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch , target_example_batch in dataset.take(1):
  example_pred = model(input_example_batch)
  print(example_pred.shape,"# batch_size , seuqence_length , vocab_size")


(64, 100, 65) # batch_size , seuqence_length , vocab_size


In [ ]:
print(len(example_pred))
print(example_pred)


64
tf.Tensor(
[[[-0.00108819 -0.00011969 -0.00149626 ...  0.00129127  0.00169046
   -0.00181506]
  [ 0.00174495  0.00432537 -0.00495144 ...  0.00388869 -0.00306225
    0.00066089]
  [-0.00072494  0.00102727  0.00330982 ...  0.00115686 -0.00336728
    0.00140809]
  ...
  [-0.00268858 -0.0062227   0.003089   ...  0.00195051 -0.00879071
    0.0032316 ]
  [-0.00075204  0.0004356   0.00074658 ...  0.00318383 -0.00829858
    0.00449194]
  [-0.00205088 -0.00578837  0.00373034 ...  0.00013473 -0.00759754
    0.00345033]]

 [[ 0.00202229  0.00377508 -0.00243458 ...  0.00222453 -0.00335881
    0.00120754]
  [-0.00555399 -0.0024911   0.00077622 ... -0.00049636 -0.00383851
   -0.00237134]
  [-0.00289193 -0.00371512 -0.00298675 ... -0.00079004  0.00251735
   -0.00528368]
  ...
  [-0.00053846  0.00735475 -0.00186109 ...  0.00679071 -0.00144884
    0.00225439]
  [-0.00611201  0.01260145  0.00107231 ...  0.00980412 -0.00047108
    0.00133855]
  [-0.01202439  0.0086741   0.00051927 ...  0.00801107  0.0

In [ ]:
pred = example_pred[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.00108819 -0.00011969 -0.00149626 ...  0.00129127  0.00169046
  -0.00181506]
 [ 0.00174495  0.00432537 -0.00495144 ...  0.00388869 -0.00306225
   0.00066089]
 [-0.00072494  0.00102727  0.00330982 ...  0.00115686 -0.00336728
   0.00140809]
 ...
 [-0.00268858 -0.0062227   0.003089   ...  0.00195051 -0.00879071
   0.0032316 ]
 [-0.00075204  0.0004356   0.00074658 ...  0.00318383 -0.00829858
   0.00449194]
 [-0.00205088 -0.00578837  0.00373034 ...  0.00013473 -0.00759754
   0.00345033]], shape=(100, 65), dtype=float32)


In [ ]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)


65
tf.Tensor(
[-1.0881887e-03 -1.1969221e-04 -1.4962550e-03  3.1207073e-03
  2.0411960e-03  3.0724853e-03 -6.0819415e-04  2.2677861e-03
  3.3034699e-04 -3.1744014e-03  1.9894186e-03 -2.1346423e-03
 -6.0013919e-03  3.6695688e-03  1.0443723e-02 -6.0492046e-03
  5.9167715e-04  2.3597470e-03 -4.5027202e-03  1.9788693e-03
 -8.5941050e-05  2.4498650e-04 -2.7856920e-03 -3.6234627e-03
  7.7233282e-03  2.3880803e-03 -1.5861066e-03  1.4620502e-03
 -3.9493176e-03 -5.0023557e-03  2.5803405e-03 -5.3438600e-03
  3.4293626e-05  4.3784478e-03  7.0142309e-04 -7.3323725e-04
 -2.8947005e-03  5.7223113e-03 -3.7392720e-03 -4.9624741e-03
  2.9963865e-03  5.8890395e-03 -3.8910853e-03 -2.3204086e-03
 -5.1123551e-03 -6.2122028e-03 -1.1151535e-03  6.3097104e-06
  3.1548548e-03  1.8876063e-03 -4.0762266e-03  2.1187186e-03
 -6.7787175e-04 -8.8218681e-04 -6.0043894e-03 -2.1485281e-03
 -3.4145964e-03 -3.2267591e-03  7.8867911e-04 -4.5985132e-03
  6.4971573e-03  1.0865321e-03  1.2912744e-03  1.6904571e-03
 -1.815057

In [ ]:
samples_indices = tf.random.categorical(pred,num_samples = 1)

samples_indices = np.reshape(samples_indices , (1,-1))[0]

pred_chars = int_to_text(samples_indices)


In [ ]:
pred_chars

'XQnooJ:3Kw;o&FvjcOzowAzDZ$$DIs\nI,$&wpwjNJXavKAo IBcq.Qy$:Rj;hA:fe?OdYX,VsRVB;3xk\n,nbLLJwLFkWo:?UikU&'

In [ ]:
def loss (labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits =True)
  

**Compiling the model**

In [ ]:
model.compile(optimizer ='adam',loss = loss)

In [ ]:
epochs = 40
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = "./training_checkpoints/ckpt_{epoch}", save_freq='epoch',
    save_weights_only=True
)


**Training the model**

In [ ]:
history = model.fit(dataset, epochs =epochs, callbacks =[checkpoint_callback])


Epoch 1/40
172/172 [==============================] - 15s 70ms/step - loss: 1.6103
Epoch 2/40
172/172 [==============================] - 14s 72ms/step - loss: 1.4866
Epoch 3/40
172/172 [==============================] - 15s 75ms/step - loss: 1.4110
Epoch 4/40
172/172 [==============================] - 17s 75ms/step - loss: 1.3567
Epoch 5/40
172/172 [==============================] - 16s 75ms/step - loss: 1.3126
Epoch 6/40
172/172 [==============================] - 16s 74ms/step - loss: 1.2732
Epoch 7/40
172/172 [==============================] - 16s 72ms/step - loss: 1.2372
Epoch 8/40
172/172 [==============================] - 14s 74ms/step - loss: 1.2019
Epoch 9/40
172/172 [==============================] - 16s 76ms/step - loss: 1.1654
Epoch 10/40
172/172 [==============================] - 15s 74ms/step - loss: 1.1294
Epoch 11/40
172/172 [==============================] - 14s 74ms/step - loss: 1.0913
Epoch 12/40
172/172 [==============================] - 15s 73ms/step - loss: 1.0521
E

**Load the model**

In [ ]:
model = build_model(VOCAB_SIZE,DIM,RNN_UNITS,batch_size =1)


In [ ]:
model.load_weights(tf.train.latest_checkpoint('./training_checkpoints'))
model.build(tf.TensorShape([1,None]))



In [ ]:
def generate_text(model,start_string):
  num_generate = 800
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  


  text_generated = []


  temp =1.0

  model.reset_states()
  for i in range(num_generate):
    pred = model(input_eval)

    pred = tf.squeeze(pred,0)

    pred = pred / temp

    pred_id = tf.random.categorical(pred ,num_samples= 1)[-1,0].numpy()

    input_eval = tf.expand_dims([pred_id],0)
    text_generated.append(idx2char[pred_id])

  return (start_string +''.join(text_generated))


In [ ]:
inp = input('Type something:')
print(generate_text(model,inp))

Type something:romeo
romeof my friends; I'll be your butchery grower.

GLOUCESTER:
Why, there thou hast twhat fatal still.
Courage, my masters! honour now of Junish wars,
His head is but one being off; see them dead:
Or, with the hazard of he stands for what; it was
ever
tender your extremes; Richard and Glordon
After the Duke of Norfolk, Thomas tapster? let's to't.

BRUTUS:
Not to them compension to unfold.

LORD WILLOUGHBY:
Be not so many more towns and be;
His hand that was; which was done a husband;
I am not married. Well and good sir: for that I have prevented them;
And, though I with a goodly son, which thou didst confess
A knature to comm;
And welcome, Juliet.

ARCHIDAMUS:
I think she too, and but a sight o'erwick grieves,
For much that way they chop against the towns.

QUEEN MARGARET:
Thanks, gentle Warwick
